# Opening a new vegan restaurant in Chicago

## Introduction/Business Problem

There has been a slow shift in interests to a vegan lifestlyle in the Chicago area.  In general the focus on being healthier has become of greater and greater concern.  Even those who do not want to eat vegan 100% of the time have seen a need to eat healhier more often.  The problem though is there seems to be a lack of vegan restaurants in the Chicago area.  The potential to take advantage of this growing trend could prove to be quite profitable.  Therefore to the right stakehoulder it would prove beneficial to examine the city of Chicago's neighborhoods to ascertain the best possible locations for success and profitability.  It is then the goal of this project to come up with the top 2 locations it seems likely a new vegan restaraunt would succeed in Chicago.

## Data Process

The data acquisition of the data will come from the the Foursquare database.  It is currently the most reliant and up to date database for this type of project.  The initial radius will be set to 1500 do to the fact lake Michigan covers a significant part of this radius.  This may begin to include some of the suburbs but by doing so there is a hope it will deliver the best possible solutions for the stakehoulders.  In effort to do this the data we will be looking for will be focused on where there are current vegan restaurants since those are areas have already shown a demand for the cuisine.

Once the data has been aquired, the data will be cleaned and analyzed to reveal the best possible solutions based on the following criteria.
    
    1. Vegan restaurants in the area
    2. Ratings of vegan restaurants in each area